In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import backend as K
import glob

#from IPython.display import Image
from PIL import Image

In [33]:
# Preprocess dataset
data_train = glob.glob("C:/Users/Adam/Desktop/COSC581_A3/train/*.jpg")
data_test = glob.glob("C:/Users/Adam/Desktop/COSC581_A3/val/*.jpg")

y_train_age = pd.read_csv("fairface_label_train.csv").loc[:, "age"]
y_test_age = pd.read_csv("fairface_label_val.csv").loc[:, "age"]
y_train_gender = pd.read_csv("fairface_label_train.csv").loc[:, "gender"]
y_test_gender = pd.read_csv("fairface_label_val.csv").loc[:, "gender"]
y_train_race = pd.read_csv("fairface_label_train.csv").loc[:, "race"]
y_test_race = pd.read_csv("fairface_label_val.csv").loc[:, "race"]

x_train = []
x_test = []

for d in data_train[0:1000]:
    i = Image.open(d).convert('RGB')
    im = np.asarray(i)
    x_train.append(im)
    
for d in data_test[0:1000]:
    i = Image.open(d).convert('RGB')
    im = np.asarray(i)
    x_test.append(im)
    
x_train = np.array(x_train)
x_test = np.array(x_test)

y_ta = np.array(y_train_age[0:1000])
y_tg = np.array(y_train_gender[0:1000])
y_tr = np.array(y_train_race[0:1000])

In [34]:
# Min-Max scaling
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

#Image(images[0])

#x_train = np.array(x_train)
#x_test = np.array(x_test)
#y_train_age[0:100] = np.array(y_train_age[0:100])
#y_test_age = np.array(y_test_age)
#y_train_gender[0:100] = np.array(y_train_gender[0:100])
#y_test_gender = np.array(y_test_gender)
#y_train_race[0:100] = np.array(y_train_race[0:100])
#y_test_race = np.array(y_test_race)

In [35]:
# Fully Connected Network
T1 = Sequential()

T1.add(layers.Flatten(input_shape=(32,32,3)))
T1.add(layers.Dense(units=1024, activation="tanh"))
T1.add(layers.Dense(units=512, activation="sigmoid"))
T1.add(layers.Dense(units=100, activation="softmax"))

# Race classifier
T1.add(layers.Dense(units=7, activation="softmax"))

# Gender classifier
#T1.add(layers.Dense(units=2, activation="softmax"))

# Age classifier
#T1.add(layers.Dense(units=9, activation="softmax"))

print(T1.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 1024)              3146752   
_________________________________________________________________
dense_20 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_21 (Dense)             (None, 100)               51300     
_________________________________________________________________
dense_22 (Dense)             (None, 7)                 707       
Total params: 3,723,559
Trainable params: 3,723,559
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
print(x_train.shape)
print(y_tr.shape)

(1000, 32, 32, 3)
(1000,)


In [37]:
opt = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
T1.compile(loss="categorical_crossentropy", optimizer=opt)

history = T1.fit(x_train, y_tr, epochs=20, batch_size=8) # y_train_race

Epoch 1/20


ValueError: in user code:

    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
       

In [38]:
# CNN
T2 = Sequential()

T2.add(layers.Conv2D(filters=40, kernel_size=(5,5), activation="relu", strides=(1,1), padding="same", input_shape=(32,32,1)))
T2.add(layers.MaxPool2D(pool_size=(2,2)))
T2.add(layers.Flatten())
T2.add(layers.Dense(units=100, activation="relu"))

# Race classifier
#T2.add(layers.Dense(units=7, activation="softmax"))

# Gender classifier
#T2.add(layers.Dense(units=2, activation="softmax"))

# Age classifier
T2.add(layers.Dense(units=9, activation="softmax"))

print(T2.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 40)        1040      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 40)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 10240)             0         
_________________________________________________________________
dense_23 (Dense)             (None, 100)               1024100   
_________________________________________________________________
dense_24 (Dense)             (None, 9)                 909       
Total params: 1,026,049
Trainable params: 1,026,049
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
opt = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
T2.compile(loss="categorical_crossentropy", optimizer=opt)

history = T2.fit(x_train, y_tr, epochs=20, batch_size=8) # y_train_race

Epoch 1/20


ValueError: in user code:

    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
       

In [40]:
# Custom CNN - LeNet-5

T3 = Sequential()

# NEED TO CHOOSE PARAMS
T3.add(layers.Conv2D(filters=6, kernel_size=(5,5), activation="sigmoid", input_shape=(32,32,1)))
T3.add(layers.AveragePooling2D(pool_size=(2,2)))
T3.add(layers.Conv2D(filters=16, kernel_size=(5,5), activation="sigmoid"))
T3.add(layers.AveragePooling2D(pool_size=(2,2)))
T3.add(layers.Flatten())
T3.add(layers.Dense(units=120, activation="sigmoid"))
T3.add(layers.Dense(units=84, activation="sigmoid"))
T3.add(layers.Dense(units=10, activation="sigmoid"))

# Race classifier
T3.add(layers.Dense(units=7, activation="softmax"))

# Gender classifier
#T3.add(layers.Dense(units=2, activation="softmax"))

# Age classifier
#T3.add(layers.Dense(units=9, activation="softmax"))

print(T3.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d (AveragePo (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_1 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 120)               48120     
_________________________________________________________________
dense_26 (Dense)             (None, 84)               

In [41]:
opt = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)
T3.compile(loss="categorical_crossentropy", optimizer=opt)

history = T3.fit(x_train, y_tr, epochs=20, batch_size=8) # y_train_race

Epoch 1/20


ValueError: in user code:

    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Adam\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
       

In [5]:
# Parallel CNN
T4 = Sequential()

In [19]:
# VAE
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(z_log_var) * epsilon

latent_dim = 8 # NEEDS TO BE AT LEAST 5
inputs = keras.Input(shape=(32,32,1))
# FIX ENCODER LAYER PARAMS
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)
z = layers.Lambda(sampling)([z_mean, z_log_var])

encoder = keras.models.Model(inputs, [z_mean, z_log_var, z])
print(encoder.summary())

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 32, 32, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 16, 16, 32)   320         input_14[0][0]                   
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 8, 8, 64)     18496       conv2d_24[0][0]                  
__________________________________________________________________________________________________
dense_21 (Dense)                (None, 8, 8, 8)      520         conv2d_25[0][0]                  
_______________________________________________________________________________________

In [23]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(8*8*64, activation="relu")(latent_inputs)
x = layers.Reshape((8,8,64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(1, 3, activation="sigmoid", padding="same")(x)

decoder = keras.models.Model(latent_inputs, decoder_outputs)
print(decoder.summary())

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 8)]               0         
_________________________________________________________________
dense_25 (Dense)             (None, 4096)              36864     
_________________________________________________________________
reshape_2 (Reshape)          (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 16, 16, 64)        36928     
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 32, 32, 32)        18464     
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 32, 32, 1)         289       
Total params: 92,545
Trainable params: 92,545
Non-trainable params: 0
__________________________________________________